In [ ]:
import cv2
import sys
import numpy as np
import math


def nothing(x):
    pass

#function for harris corner detection and localization
def harris_corner(image1,sigma,k,windowsize,threshold) :
        gray = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
        gray_image = np.float32(gray)
        rows = gray_image.shape[0]
        cols = gray_image.shape[1]
        dx = cv2.Sobel(gray_image,cv2.CV_64F,1,0,ksize=5)
        dy = cv2.Sobel(gray_image,cv2.CV_64F,0,1,ksize=5)
        ksize = max(5,  5 * sigma)
        if (ksize %2 == 0 ) :
            ksize = ksize + 1
        Ixx = cv2.GaussianBlur(dx*dx,(ksize,ksize),sigma)
        Ixy = cv2.GaussianBlur(dx*dy,(ksize,ksize),sigma)
        Iyy = cv2.GaussianBlur(dy*dy,(ksize,ksize),sigma)
        cov = np.zeros((rows,cols * 3), dtype = np.float32)
        dst = np.zeros((rows,cols), dtype = np.float32)
        for i in range(0, rows,1) :
            for j in range(0, cols,1) :
                a = cov[i, j*3] =   Ixx[i,j]
                b = cov[i, j*3+1] = Ixy[i,j]
                c = cov[i, j*3+2] = Iyy[i,j]
                dst[i,j] = a*c - b*b - k*(a+c)*(a+c)
        ret, dst1 = cv2.threshold(dst,0.001 * threshold *dst.max(),255,0)
        dst2 = np.uint8(dst1)
        ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst2)
        centroids = np.float32(centroids)
        #localization of corners to get more accuracy
        output = np.zeros((centroids.shape[0]-1,centroids.shape[1],1), dtype=np.float32 )
        window_width = windowsize * 2 + 1
        window_height = windowsize * 2 + 1
        x = np.arange(-windowsize, windowsize +1, dtype=np.int)
        y = np.arange(-windowsize, windowsize +1, dtype=np.int)
        print(len(centroids))
        for i in range(1,len(centroids)) :
            im = cv2.getRectSubPix(gray_image,(window_width, window_height), (centroids[i][0],centroids[i][1]))
            dx = cv2.Sobel(im,cv2.CV_64F,1,0,ksize=5)
            dy = cv2.Sobel(im,cv2.CV_64F,0,1,ksize=5)
            Ixx = dx**2
            Ixy = dy*dx
            Iyy = dy**2
            Sxx = np.sum(Ixx,None)
            Sxy = np.sum(Ixy,None)
            Syy= np.sum(Iyy,None)
            bb1 = Ixx * x + Ixy * y
            bb2 = Ixy * x + Iyy * y
            det = Sxx*Syy - Sxy**2
            scale =  1/det
            output[i-1][0] = centroids[i][0] + Syy*scale* np.sum(bb1,None) - Syy*scale * np.sum(bb2,None)
            output[i-1][1] = centroids[i][1] - Sxy*scale* np.sum(bb1,None) + Sxx*scale * np.sum(bb2,None)
        return output


#function for featurevector
def apply_featurevector(corners, img, windowsize):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray_image = np.float32(gray)
    width_window = windowsize * 2 + 1
    height_window = windowsize * 2 + 1
    degree_hist = np.zeros((len(corners),9,1), dtype=np.uint8)
    for i in range(0,len(corners)) :
        im = cv2.getRectSubPix(gray_image,(width_window, height_window), (corners[i][0],corners[i][1]))
        # 1st derivative of image
        dx = cv2.Sobel(im,cv2.CV_64F,1,0,ksize=5)
        dy = cv2.Sobel(im,cv2.CV_64F,0,1,ksize=5)
        angle = np.arctan2(dy,dx)
        for j in range(0, 9) :
            degree_hist[i][j] = ((( math.pi/4 * (j-4))<= angle) & (angle < (math.pi/4 *(j-3)))).sum()
    return degree_hist
   

#main function to get image, get values of required parameters, apply harris and feature vector function and print the output
def main():
    global image1,image2,var,windowsize,trace,threshold
    image1 = cv2.imread('test2.png')
    image2 = cv2.imread('test2.png')
    cv2.imshow('image1',image1)
    cv2.imshow('image2',image2)
    var = input("Variance value:")
    windowsize = input("Windowsize:")
    trace = input("Trace value:")
    threshold = input("Threshold value:")
    cv2.imshow('image1',image1)
    cv2.imshow('image2',image2)
    while True:
        key = cv2.waitKey(0)
        if key == ord('c') :
                var = 0.01 * var
                trace = 0.01 * trace
                corner_image1 = harris_corner(image1,var,trace,windowsize,threshold)
                corner_image2 = harris_corner(image1,var,trace,windowsize,threshold)
                hist_image1 = apply_featurevector(corner_image1, image1, windowsize)
                hist_image2 = apply_featurevector(corner_image2, image1, windowsize)
                c1 = np.int0(corner_image1)
                c2 = np.int0(corner_image2)
                no = 1
                min_distance = 0
                min_index = 0
                for i in range(0, len(c1)):
                    for j in range(0, len(c2)) :
                        distance = np.sum( (hist_image1[i] - hist_image2[j] ) * (hist_image1[i] - hist_image2[j] ),None)
                        if (j ==0 ) :
                            min_distance = distance
                            min_index = j
                        else  :
                            if distance < min_distance :
                                min_distance = distance
                                min_index = j
                    cv2.putText(image1,str(no), (c1[i,0],c1[i,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,255),2)
                    cv2.putText(image2,str(no), (c2[min_index,0],c2[min_index,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,255),2)
                    no = no +1
                cv2.imshow("image1",image1)
                cv2.imshow("image2",image2)
        if key == ord('h') :
            print("press 'c' for corner detection")

if __name__ == '__main__':
    main()



Variance value:0
Windowsize:5
Trace value:4
Threshold value:25
41
41
